In [1]:
import pandas as pd
import numpy as np
np.random.seed(104)
df_sample = pd.read_csv('data/MDA_09_coffee_dataset.csv')
print(df_sample.info())
print(df_sample.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2974 entries, 0 to 2973
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        2974 non-null   int64  
 1   age            2974 non-null   object 
 2   drinks_coffee  2974 non-null   bool   
 3   height         2974 non-null   float64
dtypes: bool(1), float64(1), int64(1), object(1)
memory usage: 72.7+ KB
None
   user_id   age  drinks_coffee     height
0     4509   <21          False  64.538179
1     1864  >=21           True  65.824249
2     2060   <21          False  71.319854
3     7875  >=21           True  68.569404
4     6254   <21           True  64.020226


In [2]:
iterationNum = 1000
diffHeightList = []
for _ in range(iterationNum):
bootSample = df_sample.sample(200, replace=True) # sampling with replacement
# Avg. height of people who don't drink coffee
nonCoffeeHeightMean = bootSample[bootSample['drinks_coffee'] == False].height.mean()
# Avg. height of people who drink coffee
coffeeHeightMean = bootSample[bootSample['drinks_coffee'] == True].height.mean()
diff = nonCoffeeHeightMean - coffeeHeightMean
diffHeightList.append(diff)

IndentationError: expected an indented block (2979785765.py, line 4)